In [1]:
!pip install transformers datasets torch torchvision timm evaluate

In [27]:
from datasets import load_dataset

dataset = load_dataset("kiril-buga/airplane-parts")
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 207
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 60
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 30
    })
})

In [28]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [32]:
ex = dataset['train']
dataset.features

AttributeError: 'DatasetDict' object has no attribute 'features'

## Load the Pre-trained Model and Fine-tune
### Here’s an example for fine-tuning DETR:

In [10]:
import evaluate  # New library for metrics
from transformers import DetrForObjectDetection, DetrFeatureExtractor, Trainer, TrainingArguments, DetrImageProcessor, \
    AutoImageProcessor, AutoModelForObjectDetection

# Load model and feature extractor
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

# Prepare your dataset for training
def preprocess(example_batch):
    # Use feature extractor to preprocess images and annotations
    inputs = processor(images=example_batch['image'], annotations=example_batch['annotations'], return_tensors="pt")
    return inputs

# Apply preprocessing function
train_dataset = dataset['train'].map(preprocess, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./detr-airplane-parts",
    per_device_train_batch_size=4,  # adjust based on your hardware
    per_device_eval_batch_size=4,
    num_train_epochs=10,  # adjust based on your needs
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="epoch"
)

# Load the metric using evaluate
metric = evaluate.load("mean_iou")

# Define compute metrics function
def compute_metrics(p):
    return metric.compute(predictions=p.predictions, references=p.label_ids)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Map:   0%|          | 0/100 [00:00<?, ? examples/s]


KeyError: 'annotations'